In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, get_peft_config, PeftModelForCausalLM
import torch

In [ ]:
train_path = 'data/20_000_examples_train.pt'
eval_path = 'data/20_000_examples_eval.pt'
tokenizer = AutoTokenizer.from_pretrained('AI-Sweden-Models/gpt-sw3-6.7b')

train_data = torch.load(train_path)
eval_data = torch.load(eval_path)



In [ ]:
# Load huggingface datset
from datasets import load_dataset
dataset = load_dataset("teknium/OpenHermes-2.5")

In [ ]:
# Save dataset as jsonl
dataset['train'].to_json('OpenHermes-2.5.jsonl', orient='records', lines=True)


In [ ]:
import torch
from transformers import pipeline, StoppingCriteriaList, StoppingCriteria

device = "cuda" if torch.cuda.is_available() else "cpu"


# (Optional) - define a stopping criteria
# We ideally want the model to stop generate once the response from the Bot is generated
class StopOnTokenCriteria(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id

    def __call__(self, input_ids, scores, **kwargs):
        return input_ids[0, -1] == self.stop_token_id


pipe = pipeline(
    task="text-generation",
    model="AI-Sweden-Models/gpt-sw3-6.7b-v2-translator",
    device=device
)

stop_on_token_criteria = StopOnTokenCriteria(stop_token_id=pipe.tokenizer.bos_token_id)
text = "I like to eat ice cream in the summer."

# This will translate English to Swedish
# To translate from Swedish to English the prompt would be:
# prompt = f"<|endoftext|><s>User: Översätt till Engelska från Svenska\n{text}<s>Bot:"

prompt = f"<|endoftext|><s>User: Översätt till Svenska från Engelska\n{text}<s>Bot:"

input_tokens = pipe.tokenizer(prompt, return_tensors="pt").input_ids.to(device)
max_model_length = 2048
dynamic_max_length = max_model_length - input_tokens.shape[1]

response = pipe(
    prompt,
    max_length=dynamic_max_length,
    truncation=True,
    stopping_criteria=StoppingCriteriaList([stop_on_token_criteria])
)

print(response[0]["generated_text"].split("<s>Bot: ")[-1])


In [ ]:
import torch
from transformers import pipeline, StoppingCriteriaList, StoppingCriteria, AutoModelForCausalLM


# Save model
model = AutoModelForCausalLM.from_pretrained("AI-Sweden-Models/gpt-sw3-6.7b-v2-translator")
model.save_pretrained("gpt-sw3-6.7b-v2-translator")

In [ ]:
# Upload model to hub

from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("/mnt/pr_SharedNLU/users/tim_olsen/instruction-tuning-nordic/models/qlora-test-gpt-sw3-6.7b-v2-20k-8batch-0.85split/checkpoint-3450")
tokenizer = AutoTokenizer.from_pretrained("/mnt/pr_SharedNLU/users/tim_olsen/instruction-tuning-nordic/models/qlora-test-gpt-sw3-6.7b-v2-20k-8batch-0.85split/checkpoint-3450")

model.push_to_hub("skvarre/qlora-4bit-test-gpt-sw3-6.7b")
tokenizer.push_to_hub("skvarre/qlora-4bit-test-gpt-sw3-6.7b")

In [4]:
# Merge LoRA adapters with base model and upload to hub
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import get_peft_model, get_peft_config, PeftModelForCausalLM, PeftModel 
import torch

base_model_path = "../models/gpt-sw3-6.7b-v2"
adapter_path = "./models/gpt-sw3-6.7b-qlora-4bit-openhermes-28k/checkpoint-528"
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,                     # Load model in 4-bit mode
    bnb_4bit_use_double_quantization=True, # Nested quantization 
    bnb_4bit_quant_type="nf4",             # Quantization algorithm to use 
    bnb_4bit_compute_dtype=torch.bfloat16  # data type of model after quantization
)

base_model = AutoModelForCausalLM.from_pretrained(
            base_model_path,
            torch_dtype=torch.bfloat16,
            # quantization_config=quantization_config,
            local_files_only=False,
            device_map="auto"
)

model_to_merge = PeftModel.from_pretrained(
            base_model,
            adapter_path
)

Loading checkpoint shards: 100%|██████████| 6/6 [01:45<00:00, 17.58s/it]
/home/edgelab/Tim/miniconda3/envs/pytorch-env/lib/python3.12/site-packages/peft/tuners/lora/layer.py:1059: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [5]:
merged_model = model_to_merge.merge_and_unload()

merged_model.save_pretrained("gpt-sw3-6.7b-v2-QLoRA-openhermes-28k-merged")
tokenizer.save_pretrained("gpt-sw3-6.7b-v2-QLoRA-openhermes-28k-merged")

In [ ]:
merged_model.push_to_hub("skvarre/gpt-sw3-6.7b-v2-QLoRA-openhermes-28k-merged")
# Upload tokenizer to hub as well
tokenizer.push_to_hub("skvarre/gpt-sw3-6.7b-v2-QLoRA-openhermes-28k-merged")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, get_peft_config, PeftModelForCausalLM
import torch
# Save model
model = AutoModelForCausalLM.from_pretrained("skvarre/gpt-sw3-6.7b-v2-qlora-4bit-openhermes-28k-merged")
tokenizer = AutoTokenizer.from_pretrained("skvarre/gpt-sw3-6.7b-v2-qlora-4bit-openhermes-28k-merged")

model.save_pretrained("./gpt-sw3-6.7b-v2-qlora-4bit-openhermes-28k-merged")
tokenizer.save_pretrained("./gpt-sw3-6.7b-v2-qlora-4bit-openhermes-28k-merged")